# RChain Python gRPC client 

Available at http://j.mp/rchain-py-grpc-walk-through

## Introduction

### How to reproduce this demo

- Interactive locally
- Read-only mode on the web

#### Locally on your workstation 
with Docker containers

```bash
git clone https://github.com/proof-media/rchain-notebook.git demo
cd demo
docker-compose up
```

Then open http://localhost:8888

In [3]:
# password is set in the `.env` file
# and shared to notebook container
%env NOTEBOOK_PASSWORD

'prooflikesrchainalot'

#### Read only mode

Through https://nbviewer.jupyter.org/

- [link to notebook](https://github.com/proof-media/rchain-notebook/blob/testing/name-registry/notebooks/walk-through.ipynb)
- [link to slides](https://nbviewer.jupyter.org/format/slides/github/proof-media/rchain-notebook/blob/testing%2Fname-registry/notebooks/walk-through.ipynb#/)

### Proof platform

![proof image](https://i0.wp.com/proofmedia.io/wp-content/uploads/2018/06/Default-Logo-copyxhdpi.png?resize=246%2C159&ssl=1)

**Because Truth Matters**

https://proofmedia.io/

> We are building a platform which leverages the wisdom of the crowds to establish truth.

Our backend is based on `Python 3.7+` and `Django framework`

### Credits

- inspired by the official rchain/python snippet 

https://github.com/rchain/rchain/tree/dev/node-client


- developed mainly by beetleman

https://github.com/proof-media/rchain-grpc/graphs/contributors

- I'm Paolo - CTO @Proof
- Pythonista 
- Python teacher using Jupyter Notebooks

### Roadmap and status

* Up to `0.0.10` -> code developed tested with `RNode 0.6.x`
* Upcoming `0.0.11` release will be compatible with `RNode 0.7.x`

- Current [update PR](https://github.com/proof-media/rchain-grpc/pull/10) blocked by [protobuf issue](https://github.com/protocolbuffers/protobuf/issues/5272)

- Eval/Repl [not working](https://github.com/proof-media/rchain-grpc/issues/12) for gRPC missing definition


- Short-term goal

move the repo into `rchain` GitHub space, see [issue](https://github.com/proof-media/rchain-grpc/issues/8)

(after those previous problems are solved)

## Using the code

### Connecting

With docker-compose you are running an RNode standalone instance

In [4]:
RNODE_HOST = %env NODE_NAME

In [5]:
! apk add curl

fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/community/x86_64/APKINDEX.tar.gz
(1/1) Installing curl (7.61.1-r0)
7  0%                                                                           87100% ██████████████████████████████████████████████████████████████████████████8Executing busybox-1.28.4-r1.trigger
OK: 36 MiB in 48 packages


In [6]:
! curl $RNODE_HOST:40403/version

RChain Node 0.7.1 (e747f905e7b8b92b8fde1686507cce3faf252b16)

Now we can open a channel through `gRPC` against an RChain node using the Python library.

NOTE: in this notebook docker image the package is already installed,

while normally you would do:
```bash
pip install rchain-grpc
```

In [56]:
from rchain_grpc import casper

connection = casper.create_connection(host=RNODE_HOST)
connection

### Write Rholang code

In a notebook you can write a file like this

In [57]:
%%file hello_world.rho

new print(`rho:io:stdout`) in {
    print!("Hello World!")
}


Overwriting hello_world.rho


Then edit it inside the notebook server,
e.g. <a href="http://localhost:8888/edit/hello_world.rho" target="_blank">link to file</a>

In [58]:
# read the file to pass the code
with open('hello_world.rho') as fh:
    rholang_code = fh.read()

In [59]:
# alternative: write directly the code with triple quoted strings
rholang_code = """
new print(`rho:io:stdout`) in {
    print!("Hello World!")
}
"""

In [60]:
rholang_code

'\nnew print(`rho:io:stdout`) in {\n    print!("Hello World!")\n}\n'

### Deploy and propose

In [61]:
casper.deploy(connection, rholang_code)

{'success': True, 'message': 'Success!'}

In [62]:
casper.propose(connection)

{'success': True, 'message': 'Success! Block 72d9ae5670... created and added.'}

Now check logs of the rnode to see if we have `Hello World!` string

### Play with blocks

In [40]:
output = casper.get_blocks(connection, depth=1)
# NOTE: protobuf outputs are always converted into Python dictionaries by our library
output

[{'blockHash': 'f1d38c82b433c2d2754f43e06768ae414c37a1190dca9732620f8b2869dbb53a',
  'blockSize': '1340',
  'blockNumber': 3,
  'deployCount': 1,
  'tupleSpaceHash': '95f88e33b7f5aa2d9407a4e7a7e927273ab217d841d242fa2535e30f2ae018ad',
  'timestamp': 1540201268251,
  'faultTolerance': -1.0,
  'mainParentHash': 'e2d91d598ed94df73c5085425d7ba53fcedd0db47c71c4a5ac8d252455a18c47',
  'parentsHashList': ['e2d91d598ed94df73c5085425d7ba53fcedd0db47c71c4a5ac8d252455a18c47'],
  'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}]

In [41]:
block_hash = output.pop().get('blockHash')
block_hash

'f1d38c82b433c2d2754f43e06768ae414c37a1190dca9732620f8b2869dbb53a'

In [42]:
block = casper.get_block(connection, block_hash=block_hash)
block

{'blockHash': 'f1d38c82b433c2d2754f43e06768ae414c37a1190dca9732620f8b2869dbb53a',
 'blockSize': '1340',
 'blockNumber': 3,
 'deployCount': 1,
 'tupleSpaceHash': '95f88e33b7f5aa2d9407a4e7a7e927273ab217d841d242fa2535e30f2ae018ad',
 'tupleSpaceDump': '@{Unforgeable(0x25408b316e2455a9d52da2bcd923ee3e0524b34776d387bdf347320a54f2bf6f)}!({5122eac8a5c99b7da8f6d40d5f3269c6844dabde0418749ece2aeedc64c27703 : (288, "secp256k1Verify", Nil, 1), 5a3fff1ed432237e779fc6aa20d7549d043cc0dd92180ebee0346229598870f8 : (228, "secp256k1Verify", Nil, 2), d6b0c603a51fbcd56db1606a0591310876f93bdbf2309c7bdbc277a080368990 : (188, "secp256k1Verify", Nil, 4), a3158195b3ca6dfb88b8f0cb0788b01f25ea11421104df071bd043ca27f2def4 : (202, "secp256k1Verify", Nil, 3), e84b0ca96139b9c5e1a58b55e9d84682318a2030eaf24923204a1473ed659b34 : (224, "secp256k1Verify", Nil, 5), eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08 : (92, "secp256k1Verify", Nil, 6)}) |\n@{Unforgeable(0xa4fd447dedfc960485983ee817632cf36d79f45fd

In [44]:
block.get('blockNumber')

3

### Context Manager

The connection provided by the library can be used in a `with` python statement

In [49]:
# all previous operations in one context

with casper.create_connection(host=RNODE_HOST) as connection:

    # deploy / propose
    casper.deploy(connection, rholang_code)
    print(casper.propose(connection))
    
    # handle output
    output = casper.get_blocks(connection, depth=1)
    block_hash = output.pop().get('blockHash')
    block = casper.get_block(connection, block_hash=block_hash)
    print(f"Current block number is {block.get('blockNumber')}\nwith hash {block_hash}")
    
# connection here is closed

{'success': True, 'message': 'Success! Block 765b8d7121... created and added.'}
Current block number is 7
with hash 765b8d712133d3aa003be310d15321e6e67e7862576c2160df59bf5a7149d053


### Interact with channels

By leveraging the `listenForDataAtName` we can specify a channel to receive the results of the new block proposed

In [227]:
output_placeholder = "your_channel_name"

In [228]:
rholang_code = f"""
{output_placeholder}!("bar")
"""

In [229]:
print(rholang_code)


your_channel_name!("bar")



In [230]:
with casper.create_connection(host=RNODE_HOST) as connection:
    block = casper.run_and_get_value_from(
        connection=connection,
        term=rholang_code, 
        # this gets replaced with a channel name
        output_placeholder=output_placeholder
    )

In [138]:
# what's used for replacing the placeholder
import secrets
ack_name = f'ack_{secrets.token_hex(30)}'
ack_name

'ack_891db0afecd473f4af77d294d64046a09602d090281ae47ca1e2d24a59f0'

In [231]:
block

{'status': 'Success',
 'blockResults': [{'postBlockData': [['bar']],
   'block': {'blockHash': '6a2a25352581d65f50672d068ae1e5db259e860817423534f94fd03a4eae5cc2',
    'blockSize': '1144',
    'blockNumber': 26,
    'deployCount': 1,
    'tupleSpaceHash': 'c6aafc3328b4b78c2b3a6062a25d048fa71803ef5f1454541a124970b9b4522b',
    'timestamp': 1540210892204,
    'faultTolerance': -1.0,
    'mainParentHash': '96840500fdddd0f38e71c3fc96b46dee8eefc42a83bde98d8a7e3eecc6bae7ec',
    'parentsHashList': ['96840500fdddd0f38e71c3fc96b46dee8eefc42a83bde98d8a7e3eecc6bae7ec'],
    'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}}],
 'length': 1}

In [232]:
results = block.get('blockResults')[:].pop()
messages = results.get('postBlockData')

In [233]:
for message in messages:
    print("Received: ", message.pop())

Received:  bar


### Name registry

`NEW!`

Using the techniques tested so far we can now write a contract, register it to Name Registry (introduced in 0.7) and get the id for lookup; so - all programmatically from Python - we can call the contract safely.

#### 1. Register a contract

In [199]:
ack_name = 'placeholder_something'

In [214]:
rholang_code = """
new newName, ack, register(`rho:registry:insertArbitrary`), stdout(`rho:io:stdout`) in
{
  register!(bundle+{*newName}, *ack)
  |
  contract newName(@msg) = {
    stdout!("Contract called with message: " ++ msg)
  }
  |
  for (@msg <- ack) {
    %s!(["From registry: ", msg])
  }
}
""" % ack_name

In [215]:
print(rholang_code)


new newName, ack, register(`rho:registry:insertArbitrary`), stdout(`rho:io:stdout`) in
{
  register!(bundle+{*newName}, *ack)
  |
  contract newName(@msg) = {
    stdout!("Contract called with message: " ++ msg)
  }
  |
  for (@msg <- ack) {
    placeholder_something!(["From registry: ", msg])
  }
}



In [216]:
# run and get the value in the block output
with casper.create_connection(host=RNODE_HOST) as connection:
    block = casper.run_and_get_value_from(connection, rholang_code, output_placeholder=ack_name)    

In [217]:
block

{'status': 'Success',
 'blockResults': [{'postBlockData': [[{'ps': [['From registry: '],
       ['rho:id:tmrfzpssys6hdjyfcd98e6ynr6i5z6xc6u1kdbxww4nhkjsrgbp6bs']]}]],
   'block': {'blockHash': 'aff262b776f9223f528f8ddc6b31bb02d510c3bbf8a44635731e8e1d7248067e',
    'blockSize': '3185',
    'blockNumber': 24,
    'deployCount': 1,
    'tupleSpaceHash': 'e7c52ef64c3419242a6ba68a7100d851b9363c652b78c551c94e5c208202f941',
    'timestamp': 1540210602753,
    'faultTolerance': -1.0,
    'mainParentHash': '9bdee658197b5c4faf0c86556d5a75bc6304da564cc78cf57b98a8ed77a7608d',
    'parentsHashList': ['9bdee658197b5c4faf0c86556d5a75bc6304da564cc78cf57b98a8ed77a7608d'],
    'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}}],
 'length': 1}

#### 2. Get ID in the code

We are looking to find a better way to handle the "post block data" 

In [218]:
post_block_data = block.get('blockResults')[:].pop().get('postBlockData')

In [219]:
post_block_data

[[{'ps': [['From registry: '],
    ['rho:id:tmrfzpssys6hdjyfcd98e6ynr6i5z6xc6u1kdbxww4nhkjsrgbp6bs']]}]]

In [220]:
list(post_block_data[0][0].values())[0][1][0]

'rho:id:tmrfzpssys6hdjyfcd98e6ynr6i5z6xc6u1kdbxww4nhkjsrgbp6bs'

In [221]:
import json
post_block_str = json.dumps(post_block_data)

In [222]:
import re
match = re.search(r"rho:id:[^\"]+",  post_block_str)
registry_id = match.group()
registry_id

'rho:id:tmrfzpssys6hdjyfcd98e6ynr6i5z6xc6u1kdbxww4nhkjsrgbp6bs'

#### 3. Lookup and call the contract

In [223]:
rholang_code = """
new return, lookup(`rho:registry:lookup`), stdout(`rho:io:stdout`) in
{
  lookup!(`%s`, *return) |
  for (myContract <- return) {
    myContract!("Proof test")
  }
}
""" % registry_id

In [224]:
print(rholang_code)


new return, lookup(`rho:registry:lookup`), stdout(`rho:io:stdout`) in
{
  lookup!(`rho:id:tmrfzpssys6hdjyfcd98e6ynr6i5z6xc6u1kdbxww4nhkjsrgbp6bs`, *return) |
  for (myContract <- return) {
    myContract!("Proof test")
  }
}



In [225]:
with casper.create_connection(host=RNODE_HOST) as connection:
    print(casper.deploy(connection, rholang_code))
    print(casper.propose(connection))

{'success': True, 'message': 'Success!'}
{'success': True, 'message': 'Success! Block 96840500fd... created and added.'}


go check the logs!

# THE END